# Import Required Libraries
Import pandas, datetime, os, and functions from ScapeFileLocations.

In [1]:
# Import Required Libraries
import pandas as pd
from datetime import datetime
import os
from ScapeFileLocations import populate_site_identification_rules, scrape_metashape_files

# Define Constants
Define constants such as MICA_DIR, P1_DIR, EXCEL_FILE, and SHEET_INDEX.

In [11]:
# Define Constants
MICA_DIR = r"M:\working_package_2\2024_dronecampaign\01_data\Micasense"
P1_DIR = r"M:\working_package_2\2024_dronecampaign\01_data\P1"
EXCEL_FILE = r"C:\Users\admin\Downloads\UPSCALE_drone_logbook(4).xlsx"  # Update this with the actual path to your Excel file
BASE_DIR  = r"M:\working_package_2\2024_dronecampaign\02_processing\metashape_projects\Upscale_Metashapeprojects"
OUTPUT_DIR =  r"M:\working_package_2\2024_dronecampaign\02_processing\metashape_projects\Upscale_Metashapeprojects\0001_project_lists"
SHEET_INDEX = 4  # Use 0-based index for the sheet in the Excel file

In [3]:
# Load the Excel file to inspect its contents
excel_info = pd.ExcelFile(EXCEL_FILE)

# Print the sheet names to identify the correct sheet index
print("Sheet names:", excel_info.sheet_names)

Sheet names: ['Gradient_and_paired_sites', 'SiteInfoDrone', 'site_information', 'drone_logbook', '2025_drone_logbook']


In [4]:
# Load the sheet into a DataFrame using the chosen sheet index
sheet_df = pd.read_excel(EXCEL_FILE, sheet_name=SHEET_INDEX)

# Print the headers of the chosen sheet
print("Headers in the chosen sheet:", sheet_df.columns.tolist())

Headers in the chosen sheet: ['date', 'month', 'site', 'pilot', 'uav', 'sn', 'gimbal', 'sensor1', 'sensor2', 'rtk_system', 'application', 'height', 'speed [m/s]', 'side_overlap', 'front_overlap', 'start', 'end', 'flight_duration (hh:mm)', 'weather', 'wind', 'remarks', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Site_list']


# Load and Clean Excel Data
Load the Excel file into a DataFrame, clean the date column, and filter relevant columns.

In [5]:
# Load and Clean Excel Data

# Define a function to clean the date column
def clean_date(date_str):
    try:
        return pd.to_datetime(date_str, dayfirst=True).strftime('%Y%m%d')  # Convert to 'YYYYMMDD' format
    except:
        return None  # Return None for invalid dates

# Load the Excel file into a DataFrame
df = pd.read_excel(EXCEL_FILE, sheet_name=SHEET_INDEX)

# Clean the 'date' column and apply the cleaning function
df['date'] = df['date'].apply(clean_date)

# Filter relevant columns and drop rows with missing values
df = df[['date', 'site', 'weather']].dropna()

# Prepare Site Identification Rules
Use the populate_site_identification_rules function and scrape_metashape_files to prepare site rules.

In [14]:
# Prepare Site Identification Rules

# Use the populate_site_identification_rules function to generate initial site rules
site_rules = populate_site_identification_rules(MICA_DIR, P1_DIR)

# If you need to scrape metashape files, provide valid directory paths
# Create the full output CSV file path
output_csv_path = os.path.join(OUTPUT_DIR, "metashape_file_paths.csv")
scrape_metashape_files(BASE_DIR, output_csv_path)

Dynamically populated site identification rules (Image Names from folders):
  Site: treenet_salgesch, Image Name: treenet_salgesch, Project Variants: ['treenet_salgesch']
  Site: sanasilva_50877, Image Name: sanasilva_50877, Project Variants: ['sanasilva_50877']
  Site: sanasilva_50845, Image Name: sanasilva_50845, Project Variants: ['sanasilva_50845']
  Site: sagno, Image Name: sagno, Project Variants: ['sagno']
  Site: stillberg, Image Name: stillberg, Project Variants: ['stillberg']
  Site: XX_temporary, Image Name: XX_temporary, Project Variants: ['XX_temporary']
  Site: lwf_visp, Image Name: lwf_visp, Project Variants: ['lwf_visp']
  Site: lwf_davos, Image Name: lwf_davos, Project Variants: ['lwf_davos']
  Site: treenet_sempach, Image Name: treenet_sempach, Project Variants: ['treenet_sempach']
  Site: lwf_schänis, Image Name: lwf_schänis, Project Variants: ['lwf_schänis']
  Site: marteloskop, Image Name: marteloskop, Project Variants: ['marteloskop']
  Site: lwf_neunkirch, Image 

# Define Function to Match Site Name to Folder Paths
Define the match_site_to_paths function to match site names to folder paths using the rules.

In [15]:
# Define Function to Match Site Name to Folder Paths
def match_site_to_paths(site_name, date_str, rules, micasense_dir=MICA_DIR, p1_dir=P1_DIR):
    """
    Matches a site name to its corresponding folder paths for multispectral and RGB images.

    Parameters:
        site_name (str): The name of the site.
        date_str (str): The date string in 'YYYYMMDD' format.
        rules (dict): A dictionary containing site identification rules.
        micasense_dir (str): The base directory for Micasense data.
        p1_dir (str): The base directory for P1 data.

    Returns:
        tuple: A tuple containing the multispectral and RGB folder paths.
    """
    multispec_path, rgb_path = "Folder not found", "Folder not found"
    for canonical, rule in rules.items():
        if site_name.strip() in rule['project_name_variants']:
            multispec_path = os.path.join(micasense_dir, rule['image_site_name'], date_str)
            rgb_path = os.path.join(p1_dir, rule['image_site_name'], date_str)
            break
    return multispec_path, rgb_path

# Generate Output DataFrame
Iterate through the cleaned DataFrame, match site names to paths, and save the output to a CSV file.

In [16]:
# Generate Output DataFrame

# Initialize an empty list to store the output data
output = []

# Iterate through each row in the cleaned DataFrame
for _, row in df.iterrows():
    date_str, site_name, weather = row['date'], row['site'], row['weather']
    
    # Match site name to folder paths using the defined function
    multispec, rgb = match_site_to_paths(site_name, date_str, site_rules)
    
    # Append the matched data to the output list
    output.append({
        "date": date_str,
        "site": site_name,
        "weather": weather,
        "multispec": multispec,
        "rgb": rgb
    })

# Convert the output list to a DataFrame
result_df = pd.DataFrame(output)

# Save the resulting DataFrame to a CSV file
result_df.to_csv("flight_log_with_paths.csv", index=False)

# Display the first few rows of the resulting DataFrame
result_df.head()

,date,site,weather,multispec,rgb
0,20250210,Pfynwald,sunny no clouds,M:\working_package_2\2024_dronecampaign\01_dat...,M:\working_package_2\2024_dronecampaign\01_dat...
1,20250311,Pfynwald,sunny no clouds,M:\working_package_2\2024_dronecampaign\01_dat...,M:\working_package_2\2024_dronecampaign\01_dat...
2,20250409,Sanasilva-50845,sunny no clouds,Folder not found,Folder not found
3,20250409,Sanasilva-50877,sunny no clouds,Folder not found,Folder not found
4,20250410,lwf_lens,sunny no clouds,M:\working_package_2\2024_dronecampaign\01_dat...,M:\working_package_2\2024_dronecampaign\01_dat...
